In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import xgboost as xgb
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())
import datetime
from sklearn.cross_validation import train_test_split
import operator
from scipy.stats import *

0it [00:00, ?it/s]
/Library/Python/2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv('./train_1.csv').fillna(0)

In [3]:
key = pd.read_csv('./key_trunc.csv')

In [4]:
sub = pd.read_csv("./sample_submission_1.csv")

In [5]:
train_melt = pd.read_csv("train_feats.csv")

In [234]:
for col in train.columns[1:]:
    train[col] = pd.to_numeric(train[col],downcast='integer')

KeyboardInterrupt: 

In [28]:
def get_median(row, last_n_days, rows_to_consider, is_it_train):
    if is_it_train == 1:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).median()[rows_to_consider:])
    else:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).median()[rows_to_consider - 1:])

In [29]:
def get_mean(row, last_n_days, rows_to_consider, is_it_train):
    if is_it_train == 1:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).mean()[rows_to_consider:]) 
    else:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).mean()[rows_to_consider - 1:])

In [30]:
def get_std(row, last_n_days, rows_to_consider, is_it_train):
    if is_it_train == 1:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).std()[rows_to_consider:])
    else:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).std()[rows_to_consider - 1:])

In [31]:
def get_min(row, last_n_days, rows_to_consider, is_it_train):
    if is_it_train == 1:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).min()[rows_to_consider:])
    else:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).min()[rows_to_consider - 1:])        

In [32]:
def get_max(row, last_n_days, rows_to_consider, is_it_train):
    if is_it_train == 1:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).max()[rows_to_consider:])
    else:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).max()[rows_to_consider - 1:])        

In [33]:
def get_mean_overall(row, rows_to_consider, is_it_train):
    if is_it_train == 1:
        return pd.Series(pd.Series(row)[1:].mean()).repeat(-1*rows_to_consider)
    else:
        return pd.Series(pd.Series(row)[1:].mean()).repeat(len(row) - rows_to_consider)

In [34]:
def get_median_overall(row, rows_to_consider, is_it_train):
    if is_it_train == 1:
        return pd.Series(pd.Series(row)[1:].median()).repeat(-1*rows_to_consider)
    else:
        return pd.Series(pd.Series(row)[1:].median()).repeat(len(row) - rows_to_consider)

In [35]:
# def get_mean_diff(row, last_n_days):
#     return pd.Series(pd.rolling_mean(np.diff(pd.Series(row)), window = last_n_days))

In [36]:
# def get_last_day_visits(row):
#     return row[1:]

In [37]:
def get_long_stats(func_type, df, df_melt, rows_to_consider, is_it_train = None, last_n_days = None):

    if func_type == "median":
        rolling_stats = df.apply(get_median, last_n_days = last_n_days, rows_to_consider = rows_to_consider,is_it_train = is_it_train, axis = 1)
        
    elif func_type == "mean":
        rolling_stats = df.apply(get_mean, last_n_days = last_n_days, rows_to_consider = rows_to_consider, is_it_train = is_it_train, axis = 1)
    
    elif func_type == "std":
        rolling_stats = df.apply(get_std, last_n_days = last_n_days, rows_to_consider = rows_to_consider, is_it_train = is_it_train, axis = 1)

    elif func_type == "min":
        rolling_stats = df.apply(get_min, last_n_days = last_n_days, rows_to_consider = rows_to_consider, is_it_train = is_it_train, axis = 1)

    elif func_type == "max":
        rolling_stats = df.apply(get_max, last_n_days = last_n_days, rows_to_consider = rows_to_consider, is_it_train = is_it_train, axis = 1)

    elif func_type == "mean_overall":
        rolling_stats = df.apply(get_mean_overall, rows_to_consider = rows_to_consider, is_it_train = is_it_train, axis = 1)

    elif func_type == "median_overall":
        rolling_stats = df.apply(get_median_overall, rows_to_consider = rows_to_consider, is_it_train = is_it_train, axis = 1)
        
    rolling_stats_df = pd.concat([df.Page, rolling_stats.apply(pd.Series)], axis = 1)  
    rolling_stats_df.columns = ["Page"] + list(df.columns[rows_to_consider:])
    
    rolling_stats_df = pd.melt(rolling_stats_df, id_vars=['Page'], var_name="date", value_name=func_type + "_" + str(last_n_days))        
    
    df_melt = pd.merge(df_melt, rolling_stats_df, on = ["Page", "date"])     
    
    return df_melt

In [ ]:
def generate_train_melt(train):
    
    train_melt = pd.melt(pd.concat([train.Page, 
                                    train.iloc[:, -60:]],
                                    axis = 1),
                                    id_vars=['Page'], 
                                    var_name="date", value_name="visits")

    train_melt = get_long_stats("mean", train, train_melt, rows_to_consider=-60, is_it_train = 1, last_n_days=30)
    train_melt = get_long_stats("mean", train, train_melt, rows_to_consider=-60, is_it_train = 1, last_n_days=60)
    print train_melt.head()
    
    train_melt = get_long_stats("median", train, train_melt, rows_to_consider=-60, is_it_train = 1, last_n_days=30)
    train_melt = get_long_stats("median", train, train_melt, rows_to_consider=-60, is_it_train = 1, last_n_days=60)
    print train_melt.head()

    train_melt = get_long_stats("std", train, train_melt, rows_to_consider=-60, is_it_train = 1, last_n_days=30)
    train_melt = get_long_stats("std", train, train_melt, rows_to_consider=-60, is_it_train = 1, last_n_days=60)
    print train_melt.head()

    train_melt = get_long_stats("min", train, train_melt, rows_to_consider=-60, is_it_train = 1, last_n_days=30)
    train_melt = get_long_stats("min", train, train_melt, rows_to_consider=-60, is_it_train = 1, last_n_days=60)
    print train_melt.head()

    train_melt = get_long_stats("max", train, train_melt, rows_to_consider=-60, is_it_train = 1, last_n_days=30)
    train_melt = get_long_stats("max", train, train_melt, rows_to_consider=-60, is_it_train = 1, last_n_days=60)
    print train_melt.head()

    train_melt = get_long_stats("mean_overall", train, train_melt, rows_to_consider=-60, is_it_train = 1, last_n_days=60)
    train_melt = get_long_stats("median_overall", train, train_melt, rows_to_consider=-60, is_it_train = 1, last_n_days=60)
    print train_melt.head()
        
    train_melt["date_num"] = train_melt.apply(get_date, axis = 1)
    train_melt["weekday"] = train_melt.apply(get_weekday, axis = 1)
    print train_melt.head()
    
    return train_melt 

def get_lang_features(train_melt):
    
    train_melt = pd.merge(train_melt, pd.concat([train.Page, train.progress_apply(get_language, axis = 1)], axis = 1), on = "Page")
    train_melt.columns = ['Page', 'date', 'visits', 'mean_30', 'mean_60', 'median_30', 'median_60', 'std_30', 'std_60', 'min_30', 'min_60', 'max_30', 'max_60', 'mean_overall', 'median_overall', 'date_num', 'weekday', 'language']
    
    lang_date_melt = train_melt[["language", "date", "mean_30", "mean_60", "median_30", "median_60", "min_30", "min_60", "max_30", "max_60"]]
    
    lang_feats = pd.DataFrame(lang_date_melt.groupby(["language", "date"], as_index = False).aggregate(np.median))

    lang_feats.columns = ["language", "date", "lang_mean_30_median", "lang_mean_60_median", "lang_median_30_median", "lang_median_60_median", "lang_min_30_median", "lang_min_60_median", "lang_max_30_median", "lang_max_60_median"]
        
    train_melt = pd.merge(train_melt, lang_feats, on = ["language", "date"])
    
    return train_melt

def get_weekday_features(train_melt):
    
    weekday_page_melt = train_melt[["weekday", "Page", "visits"]]
    
    weekday_mean_feats = pd.DataFrame(weekday_page_melt.groupby(["weekday", "Page"], as_index = False).aggregate(np.mean))
    weekday_mean_feats.columns = ["weekday", "Page", "weekday_mean_visits"]
    
    weekday_median_feats = pd.DataFrame(weekday_page_melt.groupby(["weekday", "Page"], as_index = False).aggregate(np.median))
    weekday_median_feats.columns = ["weekday", "Page", "weekday_median_visits"]
    
    weekday_feats = pd.merge(weekday_mean_feats, weekday_median_feats, on = ["weekday", "Page"])
        
    train_melt = pd.merge(train_melt, weekday_feats, on = ["weekday", "Page"])
    
    return train_melt
    
# def get_lang_mean(train_melt):
    
#     language_page_melt = train_melt[["language", "date", "visits"]]
    
#     language_feats = pd.DataFrame(language_page_melt.groupby(["language", "date"], as_index = False).aggregate(np.mean))

#     language_feats.columns = ["language", "date", "lang_mean_visits"]
        
#     train_melt = pd.merge(train_melt, language_feats, on = ["language", "date"])
    
#     return train_melt

train_melt = generate_train_melt(train)
train_melt = get_lang_features(train_melt)
train_melt = get_weekday_features(train_melt)
train_melt.to_csv("train_feats.csv", index = False)

                                                Page        date  visits  \
0            2NE1_zh.wikipedia.org_all-access_spider  2016-11-02    18.0   
1             2PM_zh.wikipedia.org_all-access_spider  2016-11-02    11.0   
2              3C_zh.wikipedia.org_all-access_spider  2016-11-02     3.0   
3         4minute_zh.wikipedia.org_all-access_spider  2016-11-02    12.0   
4  52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...  2016-11-02     5.0   

     mean_30    mean_60  
0  19.000000  21.066667  
1  39.933333  35.983333  
2   4.066667   4.983333  
3  19.066667  19.233333  
4  13.833333  10.350000  


In [545]:
def get_source(row):
    return row.Page.split["."][-1].split("_")[-1]

In [546]:
def get_language(row):
    try:
        return row.Page.split(".")[0].split("_")[-1]    
    except:
        return "en"

In [547]:
def get_month(row):
    return row.date.split("-")[1]

In [548]:
def get_date(row):
    return row.date.split("-")[2]

In [549]:
def get_weekday(row):
    return datetime.datetime.strptime(row.date, '%Y-%m-%d').date().weekday()

In [13]:
# def get_language_mean(row):
# lang_count = pd.Series(train_melt.groupby(["language"])["Page"].count()).sort_values(axis = 0, ascending = False)
# print lang_count

In [650]:
def get_lang_features_test(test_melt):
    
    test_melt = pd.merge(test_melt, pd.concat([overall_df.Page, overall_df.progress_apply(get_language, axis = 1)], axis = 1), on = "Page")
    test_melt.columns = ['Page', 'date', 'visits', 'mean_30', 'mean_60', 'median_30', 'median_60', 'std_30', 'std_60', 'min_30', 'min_60', 'max_30', 'max_60', 'mean_overall', 'median_overall', 'date_num', 'weekday', 'language']
    
    lang_date_melt = test_melt[["language", "date", "mean_30", "mean_60", "median_30", "median_60", "min_30", "min_60", "max_30", "max_60"]]
    
    lang_feats = pd.DataFrame(lang_date_melt.groupby(["language", "date"], as_index = False).aggregate(np.median))
    
    lang_feats.columns = ["language", "date", "lang_mean_30_median", "lang_mean_60_median", "lang_median_30_median", "lang_median_60_median", "lang_min_30_median", "lang_min_60_median", "lang_max_30_median", "lang_max_60_median"]
        
    test_melt = pd.merge(test_melt, lang_feats, on = ["language", "date"])
    
    return test_melt

In [643]:
def get_weekday_features_test(train_melt, test_melt):
    
    weekday_page_melt = train_melt[["weekday", "Page", "visits"]]
    
    weekday_mean_feats = pd.DataFrame(weekday_page_melt.groupby(["weekday", "Page"], as_index = False).aggregate(np.mean))
    weekday_mean_feats.columns = ["weekday", "Page", "weekday_mean_visits"]
    
    weekday_median_feats = pd.DataFrame(weekday_page_melt.groupby(["weekday", "Page"], as_index = False).aggregate(np.median))
    weekday_median_feats.columns = ["weekday", "Page", "weekday_median_visits"]
    
    weekday_feats = pd.merge(weekday_mean_feats, weekday_median_feats, on = ["weekday", "Page"])
        
    test_melt = pd.merge(test_melt, weekday_feats, on = ["weekday", "Page"], how="left")
    
    return test_melt

In [148]:
# def get_last_day_visits_test(row, last_index):
    
#     return row.iloc[last_index - 1]

In [13]:
def smape(y_pred, y_true):
        
    y_true = y_true.get_label()
    y_pred = np.array(y_pred)
        
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0

    return "smape", np.nanmean(diff)

In [14]:
def run_xgb(train, label_train, valid = None, label_valid = None):

    # Set our parameters for xgboost
    params = {}
    params['objective'] = 'reg:linear'
    params['eta'] = 0.05
    params['max_depth'] = 7
    params['silent'] = 1
    params['min_child_weight'] = 0
    params['subsample'] = 1
    params['colsample_bytree'] = 1
    params['nthread'] = 13
    params['gamma'] = 0
    params['max_delta_step'] = 0
    
    d_train = xgb.DMatrix(train, label=label_train)
    
    if valid is not None:
        d_valid = xgb.DMatrix(valid, label=label_valid)
        watchlist = [(d_train, 'train'), (d_valid, 'validation')]
    else:
        watchlist = [(d_train, 'train')]
        
    bst = xgb.train(params, d_train, 100, watchlist, early_stopping_rounds=10, verbose_eval=10, feval=smape, maximize = False)

    return bst

In [15]:
# label_train_melt = np.log1p(train_melt["visits"])
label_train_melt = train_melt["visits"]

# label_train_melt = boxcox(np.array(label_train_melt), lmbda=0.094) #lambda found from default boxcox on outlier-stripped data

# label_train_melt = [0] + np.diff(train_melt["visits"]).tolist()

train_melt.drop("visits", axis = 1, inplace = True)
train_melt.drop("date", axis = 1, inplace = True)
train_melt.drop("language", axis = 1, inplace = True)
train_melt.drop("Page", axis = 1, inplace = True)

# train_melt.date_num = pd.to_numeric(train_melt.date_num)

In [16]:
train_melt.drop("date_num", axis = 1, inplace=True)
train_melt.drop("weekday", axis = 1, inplace=True)

In [17]:
# x_train, x_valid, label_train, label_valid = train_teąst_split(train_melt, label_train_melt, test_size=0.2, random_state=4242)
# bst = run_xgb(x_train, label_train, x_valid, label_valid)

bst = run_xgb(train_melt, label_train_melt)

[0]	train-rmse:78327.6	train-smape:158.827
Multiple eval metrics have been passed: 'train-smape' will be used for early stopping.

Will train until train-smape hasn't improved in 10 rounds.
[10]	train-rmse:58218.3	train-smape:84.7287
[20]	train-rmse:48683.3	train-smape:64.7682
[30]	train-rmse:44627	train-smape:56.8817
[40]	train-rmse:42934.5	train-smape:53.1599
[50]	train-rmse:42233.7	train-smape:52.7741
Stopping. Best iteration:
[49]	train-rmse:42280.5	train-smape:52.4645



In [18]:
importance = bst.get_fscore()

importance = sorted(importance.items(), key=operator.itemgetter(1))

df_imp = pd.DataFrame(importance, columns=['feature', 'fscore'])
df_imp['fscore'] = df_imp['fscore']/df_imp['fscore'].sum()

print df_imp

                  feature    fscore
0                 mean_60  0.011863
1                  std_60  0.018873
2                  min_60  0.024805
3                  min_30  0.026692
4          median_overall  0.029388
5                 mean_30  0.033702
6        lang_max_30_mean  0.035859
7                  max_60  0.036398
8   weekday_median_visits  0.036668
9            mean_overall  0.039094
10    lang_median_30_mean  0.040712
11                 std_30  0.042329
12    lang_median_60_mean  0.044486
13              median_60  0.046104
14       lang_max_60_mean  0.048261
15      lang_mean_60_mean  0.053114
16       lang_min_30_mean  0.054732
17       lang_min_60_mean  0.055271
18              median_30  0.077379
19    weekday_mean_visits  0.080075
20                 max_30  0.081963
21      lang_mean_30_mean  0.082232


In [245]:
from sklearn import linear_model
reg = linear_model.LinearRegression()

# x_train, x_valid, label_train, label_valid = train_test_split(train_melt, label_train_melt, test_size=0.2, random_state=4242)
# # x_train = x_train.fillna(0)
# # x_valid = x_valid.fillna(0)
# reg.fit(x_train, label_train)

reg.fit(train_melt, label_train_melt)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [243]:
# prediction = reg.predict(x_valid)
prediction = reg.predict(test_df[["mean_30","mean_60","median_30","median_60","date_num", "weekday", "lang_mean_30_mean", "lang_mean_60_mean", "lang_median_30_mean", "lang_median_60_mean"]])

In [244]:
y_true = np.array(label_valid)
y_pred = np.array(prediction)

denominator = (np.abs(y_true) + np.abs(y_pred)) / 200.0
diff = np.abs(y_true - y_pred) / denominator
diff[denominator == 0] = 0.0

print np.nanmean(diff)

105.106730814


In [21]:
def get_preds(test_df):
    
    d_test = xgb.DMatrix(test_df)
    p_test = bst.predict(d_test)
    
    return pd.Series(p_test)

In [555]:
def get_overall_df():
    
    key_wide = pd.DataFrame()
    key_wide["page"] = np.unique(key.trunc_page).tolist()
    for i in np.unique(key.date).tolist():
        key_wide[i] = np.nan

    overall_df = pd.merge(train, key_wide, left_on="Page", right_on = "page", how = "inner")
    overall_df.drop("page", axis = 1, inplace = True)
    
    return overall_df

In [651]:
def generate_test_melt(overall_df):
        
    pred_index = overall_df.columns.tolist().index('2017-01-01') 
    
    test_melt = pd.melt(pd.concat([overall_df.Page, overall_df.iloc[:, pred_index:]],axis = 1),
                                id_vars=['Page'], 
                                var_name="date", value_name="visits")
    
    test_melt = get_long_stats("mean", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 30)
    test_melt = get_long_stats("mean", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 60)

    test_melt = get_long_stats("median", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 30)
    test_melt = get_long_stats("median", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 60)

    test_melt = get_long_stats("std", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 30)
    test_melt = get_long_stats("std", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 60)

    test_melt = get_long_stats("min", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 30)
    test_melt = get_long_stats("min", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 60)

    test_melt = get_long_stats("max", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 30)
    test_melt = get_long_stats("max", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 60)

    test_melt = get_long_stats("mean_overall", overall_df, test_melt, rows_to_consider = pred_index, is_it_train=0, last_n_days = 60)
    test_melt = get_long_stats("median_overall", overall_df, test_melt, rows_to_consider = pred_index, is_it_train=0, last_n_days = 60)
    
    test_melt["date_num"] = test_melt.apply(get_date, axis = 1)
    test_melt["weekday"] = test_melt.apply(get_weekday, axis = 1)

    return test_melt 

In [667]:
overall_df = get_overall_df()

test_melt = generate_test_melt(overall_df)
test_melt_safety = test_melt

test_melt = get_lang_features_test(test_melt)
test_melt_safety = test_melt

test_melt = get_weekday_features_test(train_melt, test_melt)
test_melt_safety = test_melt

test_melt.drop("visits", axis = 1, inplace=True)

In [766]:
for i in test_melt.columns.tolist():

    if i not in ["Page", "date", "language"]:
        
        test_melt['date'] = pd.to_datetime(test_melt['date'])

        sub_test_melt = test_melt[["Page", "date", i]]

        non_na_sub = sub_test_melt[np.isfinite(sub_test_melt[i])]
        last_date = non_na_sub.loc[non_na_sub['date'].idxmax()]["date"]

        last_rows = non_na_sub[non_na_sub["date"] == last_date][["Page", i]]
        sub_test_melt = pd.merge(sub_test_melt, last_rows, on="Page")

        sub_test_melt[i + "_x"].fillna(sub_test_melt[i + "_y"], inplace = True)

        test_melt[i] = sub_test_melt[i + "_x"]

mean_30
mean_60
median_30
median_60
std_30
std_60
min_30
min_60
max_30
max_60
mean_overall
median_overall
date_num
weekday
lang_mean_30_mean
lang_mean_60_mean
lang_median_30_mean
lang_median_60_mean
lang_min_30_mean
lang_min_60_mean
lang_max_30_mean
lang_max_60_mean
weekday_mean_visits
weekday_median_visits
Visits


In [770]:
test_melt.to_csv("test_feats.csv", index = False)

KeyboardInterrupt: 

In [6]:
test_melt = pd.read_csv("test_feats.csv")

In [22]:
# test_melt.date_num = pd.to_numeric(test_melt.date_num)
prediction = get_preds(test_melt[train_melt.columns.tolist()])

# prediction = invboxcox(get_preds(test_df[["mean_30","mean_60","median_30","median_60","date_num", "weekday"]]), ld = 0.094)

In [23]:
# pred_index = overall_df.columns.tolist().index("2017-01-01")

# pred_df = overall_df.iloc[:, pred_index:]
# pred_df["Page"] = overall_df.Page

# pred_df_melt = pd.melt(pred_df, id_vars=['Page'], var_name="date", value_name="Visits")        

test_melt["Visits"] = prediction
# pred_df = pd.concat([test_melt[["Page", "date"]], prediction], axis = 1)
pred_df = test_melt[["Page", "date", "Visits"]]

pred_id_visits = pd.merge(pred_df, key, left_on=["Page", "date"], right_on=["trunc_page", "date"], how="inner")[["Id", "Visits"]]

pred_id_visits.to_csv("pred_xgb_10.csv", index = False)

In [7]:
from scipy.stats import *
# bc_label, lambda_val = boxcox(np.array([i for i in label_train_melt if i > 4 and i < 1871])) #Outlier 10% to 90% only
# bc_label, lambda_val = boxcox(np.array([i for i in label_train_melt]))

bc_label = boxcox(np.array(label_train_melt), lmbda=0.094)

/System/Library/Frameworks/Python.framework/Versions/2.7/Extras/lib/python/scipy/stats/morestats.py:516: RuntimeWarning: divide by zero encountered in log
  y = where(lmbda == 0, log(x), (x**lmbda - 1)/lmbda)


In [8]:
def invboxcox(y,ld):
    if ld == 0:
        return(np.exp(y))
    else:
        return(np.exp(np.log(ld*y+1)/ld))

In [10]:
# np.percentile(label_train_melt, 90)
# np.percentile(label_train_melt, 10)
# len([i for i in label_train_melt if i > 4 and i < 1871])

# import matplotlib.pyplot as plt
# plt.hist(bc_label.tolist())
# plt.show()

# bc_label

# import scipy.special.inv_boxcox
invboxcox(bc_label, ld = 0.094)

/Library/Python/2.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """


array([ 18.,  11.,   3., ...,   0.,   0.,   0.])

In [26]:
test_melt

,Page,date,mean_30,mean_60,median_30,median_60,std_30,std_60,min_30,min_60,...,lang_mean_60_mean,lang_median_30_mean,lang_median_60_mean,lang_min_30_mean,lang_min_60_mean,lang_max_30_mean,lang_max_60_mean,weekday_mean_visits,weekday_median_visits,Visits
0,2NE1_zh.wikipedia.org_all-access_spider,2017-01-01,24.206897,25.847458,19.0,18.0,14.816447,27.692781,14.0,6.0,...,375.814387,317.489214,304.513993,202.962220,171.889284,1011.450385,1769.480119,28.000000,23.5,25.419800
1,2PM_zh.wikipedia.org_all-access_spider,2017-01-01,15.142857,15.142857,15.0,15.0,3.532165,3.532165,8.0,8.0,...,376.354450,317.569380,306.920184,203.746152,172.692689,988.376982,1752.265508,29.625000,21.5,24.008121
2,3C_zh.wikipedia.org_all-access_spider,2017-01-01,51.142857,59.600000,35.0,52.0,35.503186,40.401202,21.0,21.0,...,376.893892,316.815882,309.203475,204.276586,173.108675,963.020289,1742.460821,24.875000,20.0,29.285284
3,4minute_zh.wikipedia.org_all-access_spider,2017-01-01,26.428571,30.000000,27.0,26.0,6.778819,15.180729,13.0,13.0,...,376.919442,317.291424,309.175052,205.216884,173.516383,948.295417,1736.083722,19.222222,17.0,20.933161
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,2017-01-01,16.000000,12.903226,9.0,10.0,20.984121,11.178416,3.0,3.0,...,376.292870,317.220441,308.997901,206.367246,174.327309,934.614739,1720.142899,16.888889,16.0,19.521481
5,5566_zh.wikipedia.org_all-access_spider,2017-01-01,16.888889,16.769231,16.0,15.0,5.925463,9.817979,10.0,5.0,...,376.214873,318.242508,309.332556,207.917269,174.799499,916.382346,1701.346024,45.555556,18.0,31.673681
6,91Days_zh.wikipedia.org_all-access_spider,2017-01-01,20.529412,21.382979,20.0,20.0,4.638395,5.643593,13.0,12.0,...,376.077484,319.291045,309.669893,209.587570,175.210063,908.822645,1690.167327,16.444444,15.0,19.521481
7,A'N'D_zh.wikipedia.org_all-access_spider,2017-01-01,19.040000,14.163636,15.0,12.0,16.478471,12.415289,7.0,3.0,...,373.389447,320.658582,310.811538,211.844450,176.717467,894.242887,1542.505422,28.000000,23.5,24.008121
8,AKB48_zh.wikipedia.org_all-access_spider,2017-01-01,17.142857,17.142857,16.0,16.0,4.488079,4.488079,11.0,11.0,...,373.402331,320.754256,311.763643,213.393307,178.726912,878.099813,1506.263934,29.625000,21.5,24.008121
9,ASCII_zh.wikipedia.org_all-access_spider,2017-01-01,9.285714,9.727273,7.0,9.0,4.347961,3.797128,5.0,5.0,...,372.810768,320.148583,312.466156,214.402752,181.024953,837.444555,1454.117362,24.875000,20.0,20.938633
